In [1]:
import torch.nn as nn
import torch
    
class default_layer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(default_layer, self).__init__()
        self.seq = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3,1,1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.seq(x)  
    
class up_layer(nn.Module):
    def __init__(self,in_channels,out_channels):
        super(up_layer,self).__init__()
        self.upsample = nn.Upsample(scale_factor=2)
        self.layer = default_layer(in_channels, out_channels)
        
    def forward(self,x):
        return self.layer(self.upsample(x))

class recurrent_layer(nn.Module):
    def __init__(self, channels):
        super(recurrent_layer, self).__init__()
        self.layer = default_layer(channels, channels)

    def forward(self, x):
        return self.layer(x + self.layer(x))  
    
class recurrent_block(nn.Module):
    def __init__(self,  in_channels, out_channels):
        super(recurrent_block, self).__init__()
        self.r2c = nn.Sequential(
            recurrent_layer(out_channels),
            recurrent_layer(out_channels)
        )
        self.conv1x1 = nn.Conv2d(in_channels,out_channels,1)
        
    def forward(self, x):
        x = self.conv1x1(x)
        return x + self.r2c(x)
    
class down_block(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(down_block, self).__init__()
        self.r2c = recurrent_block(in_channels,out_channels)
        self.maxpool = nn.MaxPool2d(2,2)

    def forward(self, x):
        x = self.r2c(x)
        return self.maxpool(x), x
    
class up_block(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(up_block, self).__init__()
        self.in_channels = in_channels
        self.up_layer = up_layer(in_channels, out_channels)
        self.r2c = recurrent_block(in_channels,out_channels)

    def forward(self, x, attachment):
        x = self.up_layer(x)
        
        x = torch.cat((attachment,x),dim=1)
        x = self.r2c( x)
        
        return x
    
class Segnet(nn.Module):
    def __init__(self, classes):
        super(Segnet, self).__init__()
        
        self.down_rcl1 = down_block(3, 64)
        self.down_rcl2 = down_block(64, 128)
        self.down_rcl3 = down_block(128, 256)
        self.down_rcl4 = down_block(256, 512)
        
        self.down_rcl5 = recurrent_block(512, 1024)
        
        self.up_rcl1 = up_block(1024, 512)
        self.up_rcl2 = up_block(512, 256)
        self.up_rcl3 = up_block(256, 128)
        self.up_rcl4 = up_block(128, 64)
        self.conv1x1 = nn.Conv2d(64,classes,1)

    def forward(self, x):
        x, a = self.down_rcl1(x)
        x, b = self.down_rcl2(x)
        x, c = self.down_rcl3(x)
        x, d = self.down_rcl4(x)
        x = self.down_rcl5(x)
        
        x = self.up_rcl1(x, d)
        x = self.up_rcl2(x, c)
        x = self.up_rcl3(x, b)
        x = self.up_rcl4(x, a)
        x = self.conv1x1(x)
        
        return x

In [2]:
'''import os
from PIL import Image

def removesuffix(content, suffix):
    if content.endswith(suffix):
        content = content[:-len(suffix)]
    return content

def findFiles(rootDir, suffix):
    files = []
    for r, d, f in os.walk(rootDir):
        for file in f:
            if suffix in file:
                files.append(removesuffix(str(file), suffix))
    return files

def substringBefore(string, char):
    return string[:string.index(char)]

dataset = "val"

inputRoot = "/datasets/leftImg8bit/" + dataset + "/"
targetRoot = "/datasets/gtFine/" + dataset + "/"
inputSuffix = '_leftImg8bit.png'
targetSuffix = '_gtFine_labelIds.png'

reducedRoot = "/datasets/cityscapes/medium/" + dataset + "/"


os.makedirs(reducedRoot + "input/" )
os.makedirs(reducedRoot + "target/" )

files = findFiles(inputRoot, inputSuffix)
#targetImages = findFiles(inputRoot, targetSuffix)

i = 0;

targetSize = (512, 256)

for file in files:
    
    
    cityName = substringBefore(file, "_")
    
    input = Image.open(inputRoot + cityName + "/" + file + inputSuffix)
    target = Image.open(targetRoot + cityName + "/" + file + targetSuffix)

    input = input.resize(targetSize)
    target = target.resize(targetSize, Image.NEAREST)

    input.save(reducedRoot + "input/" + str(i).zfill(4) +  ".png")
    target.save(reducedRoot + "target/" + str(i).zfill(4) +  ".png")
    
    i += 1

'''

'import os\nfrom PIL import Image\n\ndef removesuffix(content, suffix):\n    if content.endswith(suffix):\n        content = content[:-len(suffix)]\n    return content\n\ndef findFiles(rootDir, suffix):\n    files = []\n    for r, d, f in os.walk(rootDir):\n        for file in f:\n            if suffix in file:\n                files.append(removesuffix(str(file), suffix))\n    return files\n\ndef substringBefore(string, char):\n    return string[:string.index(char)]\n\ndataset = "val"\n\ninputRoot = "/datasets/leftImg8bit/" + dataset + "/"\ntargetRoot = "/datasets/gtFine/" + dataset + "/"\ninputSuffix = \'_leftImg8bit.png\'\ntargetSuffix = \'_gtFine_labelIds.png\'\n\nreducedRoot = "/datasets/cityscapes/medium/" + dataset + "/"\n\n\nos.makedirs(reducedRoot + "input/" )\nos.makedirs(reducedRoot + "target/" )\n\nfiles = findFiles(inputRoot, inputSuffix)\n#targetImages = findFiles(inputRoot, targetSuffix)\n\ni = 0;\n\ntargetSize = (512, 256)\n\nfor file in files:\n    \n    \n    cityName

In [262]:
import torch.nn as nn
import torch

device = torch.device("cuda:7")
model = torch.load("/datasets/modelsWithoutNormalize/r2u_epoch_100.model", map_location=device)


In [336]:
from torch.utils import data
import os
import numpy as np
from PIL import Image
from torchvision import transforms

root = "/datasets/cityscapes/medium/val/"


mapping = [4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33]
#mapping = [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33]
reduced_mapping = [7,8,11,12,13,17,19,20,21,22,23,24,25,26,27,28,31,32,33,]


class cityscapes(data.Dataset):
    def __init__(self):
        self.tf = transforms.Compose(
            [
                transforms.ToTensor()
                #,transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ]
        )
        pass
    def __len__(self):
        return 500
    def __getitem__(self, index):
        name = str(index).zfill(4) + ".png"
        input = Image.open(root + "input/" + name )
        target = Image.open(root + "target/" + name )
        
        inputTensor = self.tf(input)
        
        targetTensor = torch.from_numpy(np.array(target)).int()
        
        #targetResultTensor = torch.zeros((len(mapping), 256, 512), dtype=torch.float32)
        
        masks = []

        for i in mapping:
            masks.append(targetTensor == i)
            
        return inputTensor, torch.stack(masks).float()

dst = cityscapes()
trainloader = data.DataLoader(dst, batch_size = 1, num_workers = 20)

19

In [306]:
from IPython.display import clear_output

'''
colors = torch.tensor([
    [  0,  0,  0],
    [111, 74,  0],
    [ 81,  0, 81],
    [128, 64,128],
    [244, 35,232],
    [250,170,160],
    [230,150,140],
    [ 70, 70, 70],
    [102,102,156],
    [190,153,153],
    [180,165,180],
    [150,100,100],
    [150,120, 90],
    [153,153,153],
    [153,153,153],
    [250,170, 30],
    [220,220,  0],
    [107,142, 35],
    [152,251,152],
    [ 70,130,180],
    [220, 20, 60],
    [255,  0,  0],
    [  0,  0,142],
    [  0,  0, 70],
    [  0, 60,100],
    [  0,  0, 90],
    [  0,  0,110],
    [  0, 80,100],
    [  0,  0,230],
    [119, 11, 32],
    [  0,  0,  0]
]) / 255
'''

colors = torch.tensor([
    [128, 64,128],
    [244, 35,232],
    [ 70, 70, 70],
    [102,102,156],
    [190,153,153],
    [153,153,153],
    [250,170, 30],
    [220,220,  0],
    [107,142, 35],
    [152,251,152],
    [ 70,130,180],
    [220, 20, 60],
    [255,  0,  0],
    [  0,  0,142],
    [  0,  0, 70],
    [  0, 60,100],
    [  0, 80,100],
    [  0,  0,230],
    [119, 11, 32],
    [0,0,0]
]) / 255

from time import sleep

model.train()
with torch.no_grad():
    for i, (inputs, targets) in enumerate(trainloader):
        #if loss != None and i % 50 == 0:
        inputs = inputs.to(device)
        targets = targets.to(device)

        outputs = model(inputs)

        outputs = outputs.cpu()
        targets = targets.cpu()

        
        targetValues, targetIndices = targets.max(dim=1)
        values, indices = outputs.max(dim=1)

        #targetIndices[targetValues < 0.1] = 30
        #indices[values < 0.1] = 30
        
        #noValidate = targetValues < 0.1
        #targetIndices[noValidate] = size
        #indices[noValidate] = size
       
        
        colorImage = torch.stack([colors[indices, 0 ], colors[indices, 1], colors[indices, 2]], dim=1)
        targetImage = torch.stack([colors[targetIndices, 0 ], colors[targetIndices, 1], colors[targetIndices, 2]], dim=1)

        clear_output(wait=False)
        
        
        print(indices.unique())

        print("-----", torch.sum(torch.logical_and((targetIndices == 22), (indices == 22)))  )
        print((targetIndices == indices).float().sum() / torch.prod(torch.tensor(list(targetIndices.size()))))
        print(outputs.shape, targets.shape)
        display(transforms.ToPILImage()( inputs[0] ))
        display(transforms.ToPILImage()( targetImage[0] ))
        display(transforms.ToPILImage()( colorImage[0] ))
        display(transforms.ToPILImage()( (indices == 30).float() ))
        sleep(5)
        break


torch.Size([1, 30, 256, 512])
torch.Size([1, 256, 512])


IndexError: index 22 is out of bounds for dimension 0 with size 20

In [337]:

import sklearn.metrics

size = len(mapping)

dice = np.zeros(size)
dice_count = np.zeros(size)
jaccard = np.zeros(size)
jaccard_count = np.zeros(size)
confusion_matrix = np.zeros((size, size))

model.train()
with torch.no_grad():
    for i, (inputs, targets) in enumerate(trainloader):
        if i % 100 == 0:
            print(i)
        inputs = inputs.to(device)
        targets = targets.to(device)

        outputs = model(inputs)

        outputs = outputs.cpu()
        targets = targets.cpu()


        targetValues, targetIndices = targets.max(dim=1)
        values, indices = outputs.max(dim=1)

        noValidate = targetValues < 0.1
        targetIndices[noValidate] = size
        indices[noValidate] = size
        
        flattenTarget = targetIndices.flatten()
        flattenOutput = indices.flatten()
        
        confusion_matrix += sklearn.metrics.confusion_matrix(flattenTarget, flattenOutput, labels=range(size))
        
        for k in range(size):
            targetIsClass = flattenTarget==k
            outputIsClass = flattenOutput==k
            occured = (torch.sum(outputIsClass) + torch.sum(targetIsClass))
            if occured != 0:
                dice_count[k] += 1
                dice[k] += torch.sum(flattenOutput[targetIsClass]==k)*2.0 / occured
            #else:
            #    dice[k] += 1
                
            denominator = torch.sum(torch.logical_or(targetIsClass, outputIsClass))
            
            
            if denominator != 0:
                jaccard_count[k] += 1
                jaccard[k] += torch.sum(torch.logical_and(targetIsClass, outputIsClass)) / denominator
            #else:
            #    jaccard[k] += 1
         
        
dice /= dice_count 
jaccard /= jaccard_count  



0
100
200
300
400


(19, 19)

In [333]:
from IPython.display import HTML, display
import tabulate

test = np.array(reduced_mapping)-4
stripped_matrix = confusion_matrix[test][:, test]

FP = stripped_matrix.sum(axis=0) - np.diag(stripped_matrix)  
FN = stripped_matrix.sum(axis=1) - np.diag(stripped_matrix)
TP = np.diag(stripped_matrix)
TN = stripped_matrix.sum() - (FP + FN + TP)

SE = np.nan_to_num(TP/(TP+FN))
SP = np.nan_to_num(TN/(TN+FP))
PC = np.nan_to_num(TP/(TP + FP))
F1 = np.nan_to_num(2 * (PC * SE) / (PC + SE))
ACC = (TP+TN)/(TP+FP+FN+TN)


names = [
    "static",
    "dynamic",
    "ground",
    "road",
    "sidewalk",
    "parking",
    "rail track",
    "building",
    "wall",
    "fence",
    "guard rail",
    "bridge",
    "tunnel",
    "pole",
    "polegroup",
    "traffic light",
    "traffic sign",
    "vegetation",
    "terrain",
    "sky",
    "person",
    "rider",
    "car",
    "truck",
    "bus",
    "caravan",
    "trailer",
    "train",
    "motorcycle",
    "bicycle"
]

    
def plot(table):
    display(HTML(tabulate.tabulate(torch.tensor(table), tablefmt='html')))
    
import pandas as pd

names = np.array(names)[test]
stripped_dice = dice[test]
stripped_jaccard = jaccard[test]

size = len(reduced_mapping)

frame = [np.append(names, "**Average"), 
         np.append(SE, np.sum(SE) / size), 
         np.append(SP, np.sum(SP) / size), 
         np.append(ACC, np.sum(ACC) / size), 
         np.append(F1, np.sum(F1) / size), 
         np.append(stripped_dice, np.sum(stripped_dice) / size), 
         np.append(stripped_jaccard, np.sum(stripped_jaccard) / size)]


pd.DataFrame(np.stack(frame, axis=1), columns=["Name","SE","SP","ACC","F1","Dice", "Jaccard"])

#print("AC:",precision / count)
#print("SE:",recall / count)
#print("SE:",recall / count)
#print("DC:",dice / count)
#print("JS:",jaccard / count)
#print("JS:",test / count)
#print("Result:",result / count)
#print("Result:",tnrs / 500)
#print(accs / 500)

,Name,SE,SP,ACC,F1,Dice,Jaccard
0,road,0.9855805011526195,0.9922825455464832,0.9897433078483191,0.9864523097237202,0.9412266845703126,0.9221177978515624
1,sidewalk,0.8932178516962564,0.9922510471660128,0.9871873022508881,0.876985929515681,0.6927998657226563,0.5960266723632812
2,building,0.9282519736045934,0.980228278688879,0.9687473399315498,0.929185599266671,0.8467560424804688,0.765544189453125
3,wall,0.505515611149414,0.9952158779712885,0.9918667714698922,0.4595059430550483,0.13047211911497997,0.09493608234309349
4,fence,0.4445445990094039,0.9968244592125416,0.992632144400986,0.47808130591743836,0.11173667488517341,0.0784496097774296
5,pole,0.5376012399712875,0.9983095826194798,0.9918814489321278,0.6488599248639118,0.535083251953125,0.3916888122558594
6,traffic light,0.5339232764149804,0.999717078852113,0.998828798783373,0.6348680029400224,0.30728906588230087,0.22683875326545774
7,traffic sign,0.680015884922561,0.9994750421846834,0.9975499805700889,0.7698555821423397,0.5736288464503733,0.4539547198214512
8,vegetation,0.9530504513977873,0.9830605871088663,0.9777901634129794,0.937780958606781,0.8748405013151302,0.8064182564347445
9,terrain,0.6506491944895684,0.9978158699045977,0.9952056991380466,0.6711300530766846,0.24327417021816217,0.18421856407980317


In [164]:
len(names)

30

In [95]:
r = colors[:, 0]
g = colors[:, 1]
b = colors[:, 2]

values, indices = torch.max(a, 1)

indices = torch.tensor([[[0,0.1],[2,3]]])

indices[indices < 0.3] = 255
indices

tensor([[[255., 255.],
         [  2.,   3.]]])

In [119]:
a = torch.tensor([[[[-0.1,-0.1],[2,3]], [[-0.1,0.1],[2,3]]]])

values, indices = a.max(dim=0)

indices[values < 0.1] = 19
indices

tensor([[[19, 19],
         [ 0,  0]],

        [[19,  0],
         [ 0,  0]]])

In [40]:
m = torch.tensor([[1,2,3],[4,5,6],[7,8,9]])


In [51]:
m[-1:0, 1]

tensor([], dtype=torch.int64)